In [1]:
import pandas as pd

# Path: processamento_dados.ipynb
# Ler arquivo formato xlsx
df1 = pd.read_excel('Dataset_IADE_Orders_21_23.xlsx')
df2 = pd.read_excel('Dataset_IADE_Orders19_20.xlsx')
df_produtos = pd.read_excel('Dataset_IADE_Products.xlsx')

print(df1.shape)
print(df2.shape)
print(df_produtos.shape)


(41265, 20)
(10025, 20)
(10292, 5)


In [2]:
#Juntar os dados das encomendas numa só tabela
df_orders = pd.concat([df1, df2], ignore_index=True)
df_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Row ID          51290 non-null  int64         
 1   Order ID        51290 non-null  object        
 2   Order Date      51290 non-null  datetime64[ns]
 3   Ship Date       51290 non-null  datetime64[ns]
 4   Ship Mode       51290 non-null  object        
 5   Customer ID     51290 non-null  object        
 6   Customer Name   51281 non-null  object        
 7   Segment         51290 non-null  object        
 8   City            51290 non-null  object        
 9   State           51290 non-null  object        
 10  Country         51290 non-null  object        
 11  Postal Code     9994 non-null   float64       
 12  Market          51290 non-null  object        
 13  Region          51290 non-null  object        
 14  Product ID      51290 non-null  object        
 15  Qu

In [3]:
#intendificar colunas com dados nulos ou faltantes
df_orders.isnull().sum()

Row ID                0
Order ID              0
Order Date            0
Ship Date             0
Ship Mode             0
Customer ID           0
Customer Name         9
Segment               0
City                  0
State                 0
Country               0
Postal Code       41296
Market                0
Region                0
Product ID            0
Quantity              0
Discount              0
Profit                0
Shipping Cost         0
Order Priority        5
dtype: int64

## Processamento de dados

In [4]:
#apagar coluna Postal Code de df
df_orders = df_orders.drop(['Postal Code'], axis=1)
df_orders.isnull().sum()


Row ID            0
Order ID          0
Order Date        0
Ship Date         0
Ship Mode         0
Customer ID       0
Customer Name     9
Segment           0
City              0
State             0
Country           0
Market            0
Region            0
Product ID        0
Quantity          0
Discount          0
Profit            0
Shipping Cost     0
Order Priority    5
dtype: int64

In [5]:
#Na coluna Market, subistitua o valor atual para 'Iberian' para paises de Portugal e Spain.
df_orders.loc[df_orders['Country'].isin(['Portugal', 'Spain']), 'Market'] = 'Iberian'

In [6]:
df_produtos.head()

,Product ID,Category,Sub-Category,Product Name,UnitPrice
0,FUR-ADV-10000002,Furniture,Furnishings,"Advantus Photo Frame, Duo Pack",53.04
1,FUR-ADV-10000108,Furniture,Furnishings,"Advantus Clock, Erganomic",50.01
2,FUR-ADV-10000183,Furniture,Furnishings,"Advantus Photo Frame, Black",52.98
3,FUR-ADV-10000188,Furniture,Furnishings,"Advantus Stacking Tray, Erganomic",24.99
4,FUR-ADV-10000190,Furniture,Furnishings,"Advantus Frame, Duo Pack",111.18


In [7]:
#verificar nulls em df_produtos
df_produtos.isnull().sum()

Product ID        0
Category        366
Sub-Category      0
Product Name      0
UnitPrice         0
dtype: int64

In [8]:
#Atribuir valor 'Furniture' a coluna Category onde Sub-Category = 'Tables'
df_produtos.loc[df_produtos['Sub-Category'] == 'Tables', 'Category'] = 'Furniture'
df_produtos['Category'].fillna('Furniture', inplace=True)
#Verificar valores vazios em Category

df_produtos.isnull().sum()

Product ID      0
Category        0
Sub-Category    0
Product Name    0
UnitPrice       0
dtype: int64

## Criando dimensoes clientes e cidades

In [9]:
#Novo dataframe com as colunas dos clientes
df_clientes = df_orders[['Customer ID', 'Customer Name', 'Segment']].copy()
#apagar colunas de df_orders
df_orders = df_orders.drop(['Customer Name', 'Segment', 'Row ID'], axis=1)
df_clientes.head()


,Customer ID,Customer Name,Segment
0,SC-20575,Sonia Cooley,Consumer
1,KW-16570,Kelly Williams,Consumer
2,KW-16570,Kelly Williams,Consumer
3,KW-16570,Kelly Williams,Consumer
4,KW-16570,Kelly Williams,Consumer


In [10]:
#crie um novo df_localizacao de df com as colunas Order_ID, City, Country,
df_localizacao = df_orders[['City', 'State', 'Country', 'Market', 'Region']].copy()
#apagar colunas de df_orders
df_orders = df_orders.drop(['State', 'Country', 'Market', 'Region'], axis=1)
df_localizacao.head()


,City,State,Country,Market,Region
0,Mexico City,Distrito Federal,Mexico,LATAM,North
1,Dos Quebradas,Risaralda,Colombia,LATAM,South
2,Dos Quebradas,Risaralda,Colombia,LATAM,South
3,Dos Quebradas,Risaralda,Colombia,LATAM,South
4,Dos Quebradas,Risaralda,Colombia,LATAM,South


## Remover linhas duplicadas em função da coluna de interesse

In [11]:
#remover linhas duplicadas em função de city
df_localizacao = df_localizacao.drop_duplicates(subset=['City'])
df_localizacao.head(2)

,City,State,Country,Market,Region
0,Mexico City,Distrito Federal,Mexico,LATAM,North
1,Dos Quebradas,Risaralda,Colombia,LATAM,South


In [12]:
#remover linhas duplicadas em função de Customer ID
df_clientes = df_clientes.drop_duplicates(subset=['Customer ID'])
df_clientes.head(2)

,Customer ID,Customer Name,Segment
0,SC-20575,Sonia Cooley,Consumer
1,KW-16570,Kelly Williams,Consumer


## Salvar Dataframes em xlsx

In [13]:
#exportar df para xlsx no diretorio atual na pasta processamento_dados
df_orders.to_excel('./processamento_dados/Dataset_IADE_Orders.xlsx', index=False)


In [ ]:
#exportar df_produtos para xlsx no diretorio atual
df_produtos.to_excel('./processamento_dados/Dataset_IADE_Products.xlsx', index=False)

In [ ]:
#exportar df_clientes para xlsx no diretorio atual
df_clientes.to_excel('./processamento_dados/Dataset_IADE_Customers.xlsx')

In [ ]:
#exportar df_localizacao para xlsx no diretorio atual
df_localizacao.to_excel('./processamento_dados/Dataset_IADE_Localizacao.xlsx')